In [ ]:
%pip install Keras
%pip install tensorflow

In [ ]:
from keras import models
from keras import layers
from keras import activations
from keras import initializers
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
from keras import callbacks
from keras.preprocessing import image

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D( #convolution
    32, (5, 5), #32 convolution filters 5x5 (take 4 pixels out 64x64 -> 60x60)
    input_shape = (64, 64, 3), #input, 64x64 rgb(3 entry per pixel)
    activation = 'relu'
))
model.add(layers.MaxPooling2D(
    pool_size = (2, 2) #get the biggest value in a 2x2 (60x60 -> 30x30)
))
model.add(layers.Conv2D(
    16, (5, 5), #our choice
    input_shape = (30, 30, 3),
    activation = 'relu'
)) #26x26
model.add(layers.MaxPooling2D(
    pool_size = (2, 2)
)) #13x13
model.add(layers.Conv2D(
    4, (5, 5),
    input_shape = (13, 13, 3),
    activation = 'relu'
)) #9x9
model.add(layers.MaxPooling2D(
    pool_size = (2, 2)
)) #5x5 -> round

model.add(layers.Flatten()) #img (matrix) -> vector with
model.add(layers.Dense(
    64, #quantity of neurons
    kernel_regularizer = regularizers.L2(1e-4), #apply a penalty for high weights
    kernel_initializer = initializers.RandomNormal(stddev = 1), #weights start value,
    #in this case we utilize standard deviation to find this starting values
    bias_initializer = initializers.Zeros() #bias start as 0
))
# 25*32 + 16*25 + 4*25 + 150000 imgs * 64 = 9601300 learning parameters
model.add(layers.Dropout(0.2)) #20% of the neurons will be dropped out
model.add(layers.Activation(activations.relu)) #filter if the neuron is important
#(relu maintain positive numbers and changes negative to 0)

model.add(layers.Dense(64,
  kernel_regularizer = regularizers.L2(1e-4),
  kernel_initializer = initializers.RandomNormal(stddev = 1),
  bias_initializer = initializers.Zeros()
))
model.add(layers.Activation(activations.relu))


model.add(layers.Dense(64,
  kernel_regularizer = regularizers.L2(1e-4),
  kernel_initializer = initializers.RandomNormal(stddev = 1),
  bias_initializer = initializers.Zeros()
))
model.add(layers.Activation(activations.relu))
model.add(layers.BatchNormalization())

model.add(layers.Dense(2,
  kernel_initializer = initializers.RandomNormal(stddev = 1),
  bias_initializer = initializers.Zeros()
))
model.add(layers.Activation(activations.softmax))

In [ ]:
model.compile(
    optimizer=optimizers.Adam(), #another ex.: SGD
    loss=losses.BinaryCrossentropy(),
    metrics=[metrics.Accuracy()]
)

In [ ]:
dataGen = image.ImageDataGenerator(
    rescale = 1.0/255, #0-255 -> 0-1
    shear_range = 0.2, #random cuts in the images (20%)
    zoom_range = 0.2, #random zooms in the images (20%)
    horizontal_flip = True,
    vertical_flip = False,
    #create another images from the primary images
    validation_split = 0.2 #it saves a % of the data for test (20%)
)

X_train = dataGen.flow_from_directory(
    'PetImages', #he sees the folder name and this will be the y (target)
    target_size = (64, 64), #resize the images
    batch_size = 32, #gets 32 images and do the training with them
    class_mode = 'categorical', #has to be the same as the output
    subset = 'training'
)

X_tests = dataGen.flow_from_directory(
    'PetImages',
    target_size = (64, 64), #resize the images
    batch_size = 32, #gets 32 images and do the training with them
    class_mode = 'categorical', #has to be the same as the output
    subset = 'validation'
)

In [ ]:
total_data = 32000
batchSize = 32

model.fit(X_train,
          validation_data = X_tests,
          steps_per_epoch = total_data / batchSize,
          epochs = 50, #max
          validation_steps = (total_data / batchSize) / 10, #the amount of times he will verify the test steps to give us the precision
          callbacks = [
            callbacks.EarlyStopping(patience = 4), #if the error function dont get better in 4 epochs, it stops
            callbacks.ModelCheckpoint(
                filepath = 'model.{epoch:02d}-{val_loss:.2f}.h5' #val_precision is with X_test
            )
          ]
)

model.save('model') #it saves it in a folder named model